In [272]:
import numpy as np
from keras.utils import to_categorical
from keras import models
from keras import layers
from keras import preprocessing
from nltk.tokenize import TweetTokenizer


In [273]:
import pandas as pd
import ast
from keras.preprocessing import sequence

In [309]:

from keras.layers import SimpleRNN, LSTM, GRU, Bidirectional, Dense, Embedding, Dropout, BatchNormalization
from keras.models import Sequential

In [275]:
def frequency_dictionary(tweets):

  # Create a dictionary to store word counts.
  word_count = {}
  for tweet in tweets:
    # Split the review into words and convert to lowercase.
    for word in tweet:
      word_count[word] = word_count.get(word, 0) + 1

  # Sort words by frequency (descending order)
  sorted_words = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

  # Create a dictionary to map words to integer indices based on frequency.
  word_to_index = {word: i for i, (word, _) in enumerate(sorted_words)}

  return word_to_index


In [276]:
train = pd.read_csv('processed_tweets_stemm/train.csv')
validation = pd.read_csv('processed_tweets_stemm/validation.csv')
test = pd.read_csv('processed_tweets_stemm/test.csv')

In [277]:
test

,Unnamed: 0,texto,sentimiento,texto_normalizado
0,1554,"Los funcionarios, además de tener congelado el...",1,funcionari ademas ten congel sueld trabaj dos ...
1,2300,Toda mi solidaridad con la familia d Marta del...,1,tod solidar famili d mart castillo.hoy sid dia...
2,12553,"@victorita_fest para nada, ya arranque el segu...",0,arranqu segund term veng find larg da calent b...
3,1146,Son bonos emitidos centralizadamente x el Teso...,2,bon emit centraliz x tesor europe respald conj...
4,15619,@marianorajoy Va a explicar el programa o no ?...,1,va explic program voy vot nunc prestig
...,...,...,...,...
3409,167,"Sobre ""accidentes""... RT @santigonzalez10 Él h...",1,accident ... rt lleg
3410,6931,"¿Cómo iba vestida? ""¿Cómo explicarlo? -me dice...",1,¿ com iba vest ¿ com explic dic luis b si llev...
3411,14010,@jrojasnovoa eso mismo de hecho has logrado un...,2,mism hech logr monton valor esfuerz bien envidi
3412,9828,Me duele saber que en mis relaciones soy super...,1,duel sab relacion sup fiel pendej final result...


In [278]:
train.dropna(subset=['texto_normalizado'], inplace=True)
validation.dropna(subset=['texto_normalizado'], inplace=True)
test.dropna(subset=['texto_normalizado'], inplace=True)

In [279]:
train['texto_normalizado'] = train['texto_normalizado'].apply(lambda x: x.split(' '))
validation['texto_normalizado'] = validation['texto_normalizado'].apply(lambda x: x.split(' '))
test['texto_normalizado'] = test['texto_normalizado'].apply(lambda x: x.split(' '))

train['texto_normalizado'].to_list()

[['bienven', 'trist', 'realid', 'andam', 'mism'],
 ['braz', 'novi', 'unic', 'quier', 'necesit'],
 ['pas', 'dig'],
 ['tambi',
  'empres',
  'independient',
  'contact',
  'si',
  'estub',
  'interes',
  'segur',
  'avanz'],
 ['islandi',
  'juici',
  'respons',
  'crisis',
  'negat',
  'pag',
  'deud',
  'elabor',
  'nuev',
  'constitu',
  'camin'],
 ['ton',
  'dur',
  'concesion',
  'replic',
  'rajoy',
  'amaiur',
  'anunci',
  'com',
  'encar',
  'relacion',
  'logic',
  'peor',
  '...'],
 ['comienz', 'intervencion', 'acto', 'cierr', 'campañ', 'siguel', 'direct'],
 ['bdi',
  'em',
  'ira',
  'puent',
  'si',
  'vais',
  'dejeis',
  'llev',
  'tablet',
  'pc',
  'lueg',
  'orbyt',
  'moment',
  'dig'],
 ['ff', 'demostr', 'penultim', 'tweet', 'prejuici', 'oportun', 'actualiz'],
 ['herenci',
  'trabaj',
  'rat',
  '5',
  'eur',
  'hor',
  'editorial',
  '20.30',
  'h',
  '¡',
  'graci',
  'zapater'],
 ['tumb', 'tra', 'buen', 'recuerd'],
 ['feliz', 'dia', 'inocent', 'pic', 'pel', 'sig', '

In [280]:
freq_dict = frequency_dictionary(train.texto_normalizado.to_list())

In [281]:
freq_dict

{'...': 0,
 'si': 1,
 'q': 2,
 'buen': 3,
 'hac': 4,
 'hoy': 5,
 '¿': 6,
 'graci': 7,
 'sol': 8,
 'vot': 9,
 'quier': 10,
 '20n': 11,
 '“': 12,
 'd': 13,
 'mejor': 14,
 'dia': 15,
 'pued': 16,
 'pas': 17,
 'ser': 18,
 'rt': 19,
 'nuev': 20,
 'rajoy': 21,
 'ver': 22,
 'dias': 23,
 'pp': 24,
 'ahor': 25,
 'dic': 26,
 'esper': 27,
 'bien': 28,
 'asi': 29,
 'sab': 30,
 '¡': 31,
 'part': 32,
 'va': 33,
 'gobiern': 34,
 'mas': 35,
 '”': 36,
 'trabaj': 37,
 '..': 38,
 'cre': 39,
 'mañan': 40,
 'año': 41,
 'mal': 42,
 'dej': 43,
 'españ': 44,
 'feliz': 45,
 'habl': 46,
 'tan': 47,
 'amig': 48,
 'años': 49,
 'voy': 50,
 'gan': 51,
 'cos': 52,
 'cas': 53,
 'lleg': 54,
 'noch': 55,
 'pso': 56,
 'sal': 57,
 'gust': 58,
 'tod': 59,
 'qued': 60,
 'gent': 61,
 'madr': 62,
 'polit': 63,
 'gran': 64,
 'com': 65,
 'deb': 66,
 'pod': 67,
 'siempr': 68,
 'public': 69,
 'mism': 70,
 'hor': 71,
 'vez': 72,
 'acab': 73,
 'cambi': 74,
 'much': 75,
 'derech': 76,
 'ir': 77,
 'vid': 78,
 'vam': 79,
 'sig': 80,


In [282]:
train['texto_encodeado'] = train['texto_normalizado'].apply(lambda tweet: [freq_dict.get(word, 0) for word in tweet])
validation['texto_encodeado'] = validation['texto_normalizado'].apply(lambda tweet: [freq_dict.get(word, 0) for word in tweet])
test['texto_encodeado'] = test['texto_normalizado'].apply(lambda tweet: [freq_dict.get(word, 0) for word in tweet])

train['texto_encodeado']

0                                [821, 125, 471, 3192, 70]
1                                [2055, 446, 120, 10, 110]
2                                                [17, 150]
3          [701, 351, 1144, 1859, 1, 4090, 194, 170, 1015]
4        [5865, 1297, 648, 160, 1402, 145, 379, 3193, 2...
                               ...                        
10236                      [8, 10, 13569, 13570, 557, 593]
10237    [120, 551, 106, 331, 1673, 13571, 5724, 1064, ...
10238    [1850, 454, 719, 259, 49, 1644, 572, 1053, 375...
10239                  [823, 623, 5857, 5296, 4602, 13574]
10240                  [328, 187, 236, 1372, 92, 799, 266]
Name: texto_encodeado, Length: 10194, dtype: object

In [283]:
train['texto_normalizado']

0                    [bienven, trist, realid, andam, mism]
1                       [braz, novi, unic, quier, necesit]
2                                               [pas, dig]
3        [tambi, empres, independient, contact, si, est...
4        [islandi, juici, respons, crisis, negat, pag, ...
                               ...                        
10236              [sol, quier, bailart, medus, loc, priv]
10237    [unic, pus, quer, jug, lol, tortug, retwitt, m...
10238    [audienci, nacional, conden, 20, años, prision...
10239    [socied, muer, apuñal, propietari, zapat, banyol]
10240    [sanid, educ, comun, calid, español, 20nupyd, ...
Name: texto_normalizado, Length: 10194, dtype: object

In [284]:
max_length = 0

for inner_list in train['texto_encodeado'].to_list():
  current_length = len(inner_list)
  
  if current_length > max_length:
    max_length = current_length

print("Longest word count of a tweet:", max_length)

Longest word count of a tweet: 28


In [285]:
train['texto_encodeado']

0                                [821, 125, 471, 3192, 70]
1                                [2055, 446, 120, 10, 110]
2                                                [17, 150]
3          [701, 351, 1144, 1859, 1, 4090, 194, 170, 1015]
4        [5865, 1297, 648, 160, 1402, 145, 379, 3193, 2...
                               ...                        
10236                      [8, 10, 13569, 13570, 557, 593]
10237    [120, 551, 106, 331, 1673, 13571, 5724, 1064, ...
10238    [1850, 454, 719, 259, 49, 1644, 572, 1053, 375...
10239                  [823, 623, 5857, 5296, 4602, 13574]
10240                  [328, 187, 236, 1372, 92, 799, 266]
Name: texto_encodeado, Length: 10194, dtype: object

In [286]:
len(sequence.pad_sequences(train['texto_encodeado'].to_list(), maxlen=27, padding='post').tolist())

10194

In [287]:
train['texto_encodeado'] = sequence.pad_sequences(train['texto_encodeado'].to_list(), maxlen=27, padding='post').tolist()
validation['texto_encodeado'] = sequence.pad_sequences(validation['texto_encodeado'].to_list(), maxlen=27, padding='post').tolist()
test['texto_encodeado'] = sequence.pad_sequences(test['texto_encodeado'].to_list(), maxlen=27, padding='post').tolist()

In [288]:
train['texto_encodeado']

0        [821, 125, 471, 3192, 70, 0, 0, 0, 0, 0, 0, 0,...
1        [2055, 446, 120, 10, 110, 0, 0, 0, 0, 0, 0, 0,...
2        [17, 150, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3        [701, 351, 1144, 1859, 1, 4090, 194, 170, 1015...
4        [5865, 1297, 648, 160, 1402, 145, 379, 3193, 2...
                               ...                        
10236    [8, 10, 13569, 13570, 557, 593, 0, 0, 0, 0, 0,...
10237    [120, 551, 106, 331, 1673, 13571, 5724, 1064, ...
10238    [1850, 454, 719, 259, 49, 1644, 572, 1053, 375...
10239    [823, 623, 5857, 5296, 4602, 13574, 0, 0, 0, 0...
10240    [328, 187, 236, 1372, 92, 799, 266, 0, 0, 0, 0...
Name: texto_encodeado, Length: 10194, dtype: object

In [289]:
train['sentimiento'] = train['sentimiento'].astype(float)
validation['sentimiento'] = validation['sentimiento'].astype(float)
test['sentimiento'] = test['sentimiento'].astype(float)

In [290]:
model = models.Sequential()

In [312]:
# Input - Layer
model.add(layers.Dense(50, activation = "relu", input_shape=(10000, )))
# Hidden - Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(layers.Dense(50, activation = "relu"))
# Output- Layer
model.add(layers.Dense(1, activation = "sigmoid"))
model.summary()

/home/azul/.local/lib/python3.10/site-packages/keras/src/layers/core/dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ simple_rnn_4 (SimpleRNN)        │ (None, 27, 32)            │      1,088 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ batch_normalization             │ (None, 27, 32)            │        128 │
│ (BatchNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_10 (Dropout)            │ (None, 27, 32)            │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ simple_rnn_5 (SimpleRNN)        │ (None, 32)                │      2,080 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ batch_normalization_1           │ (None, 32)                │        128 │
│ (BatchNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_11 (Dropout)            │ (None, 32)                │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_24 (Dense)                │ (None, 1)                 │         33 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_31 (Dense)                │ (None, 50)                │        100 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_16 (Dropout)            │ (None, 50)                │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_32 (Dense)                │ (None, 50)                │      2,550 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_17 (Dropout)            │ (None, 50)                │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_33 (Dense)                │ (None, 50)                │      2,550 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_34 (Dense)                │ (None, 1)                 │         51 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 15,368 (60.04 KB)

 Trainable params: 8,580 (33.52 KB)

 Non-trainable params: 128 (512.00 B)

 Optimizer params: 6,660 (26.02 KB)

## Binary model

In [293]:
train_binary = train[train['sentimiento'] != 2.0]
valid_binary = validation[validation['sentimiento'] != 2.0]
test_binary = test[test['sentimiento'] != 2.0]

train_binary

,Unnamed: 0,texto,sentimiento,texto_normalizado,texto_encodeado
0,10221,"@Manuellflorod Bienvenida (triste) realidad, a...",1.0,"[bienven, trist, realid, andam, mism]","[821, 125, 471, 3192, 70, 0, 0, 0, 0, 0, 0, 0,..."
2,7555,"Todos tenemos un pasado @ahorapodemos ,que se ...",0.0,"[pas, dig]","[17, 150, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,2825,En Islandia: juicio a los responsables de la ...,0.0,"[islandi, juici, respons, crisis, negat, pag, ...","[5865, 1297, 648, 160, 1402, 145, 379, 3193, 2..."
5,1038,El tono duro y sin concesiones de la réplica d...,1.0,"[ton, dur, concesion, replic, rajoy, amaiur, a...","[2056, 252, 3194, 2643, 21, 510, 277, 65, 2644..."
6,6331,Comienza su intervención en el acto de cierre ...,0.0,"[comienz, intervencion, acto, cierr, campañ, s...","[529, 581, 491, 492, 180, 4091, 292, 0, 0, 0, ..."
...,...,...,...,...,...
10230,16024,Preparaos! Se acerca el momento... Ese en el q...,0.0,"[prepara, acerc, moment, ..., q, q, afront, qu...","[13567, 824, 171, 0, 2, 2, 2809, 106, 1506, 45..."
10235,6266,"Irene Villa: """"El aborto no es un derecho de l...",1.0,"[iren, vill, abort, derech, muj, martiri, 20:30]","[5020, 5651, 967, 76, 519, 4487, 262, 0, 0, 0,..."
10237,11965,@cuervotinelli lo único que te puse fue que qu...,1.0,"[unic, pus, quer, jug, lol, tortug, retwitt, m...","[120, 551, 106, 331, 1673, 13571, 5724, 1064, ..."
10238,5390,La Audiencia Nacional condena a 20 años de pri...,1.0,"[audienci, nacional, conden, 20, años, prision...","[1850, 454, 719, 259, 49, 1644, 572, 1053, 375..."


In [294]:
#train_binary['texto_encodeado'] = train_binary['texto_encodeado'].apply(lambda x: np.asarray(x))
#valid_binary['sentimiento'] = valid_binary['texto_encodeado'].apply(lambda x: np.asarray(x))

In [295]:
import tensorflow as tf


In [296]:
tf.convert_to_tensor(train_binary['texto_encodeado'].to_list(), dtype=tf.int64)

<tf.Tensor: shape=(6787, 27), dtype=int64, numpy=
array([[ 821,  125,  471, ...,    0,    0,    0],
       [  17,  150,    0, ...,    0,    0,    0],
       [5865, 1297,  648, ...,    0,    0,    0],
       ...,
       [ 120,  551,  106, ...,    0,    0,    0],
       [1850,  454,  719, ...,    0,    0,    0],
       [ 823,  623, 5857, ...,    0,    0,    0]])>

In [297]:
x_train_tensor = tf.convert_to_tensor(train_binary['texto_encodeado'].to_list(), dtype=tf.int64)
x_valid_tensor = tf.convert_to_tensor(valid_binary['texto_encodeado'].to_list(), dtype=tf.int64)
x_test_tensor = tf.convert_to_tensor(test_binary['texto_encodeado'].to_list(), dtype=tf.int64)

In [298]:
test_binary

,Unnamed: 0,texto,sentimiento,texto_normalizado,texto_encodeado
0,1554,"Los funcionarios, además de tener congelado el...",1.0,"[funcionari, ademas, ten, congel, sueld, traba...","[1298, 746, 107, 1248, 681, 37, 95, 71, 136, 8..."
1,2300,Toda mi solidaridad con la familia d Marta del...,1.0,"[tod, solidar, famili, d, mart, castillo.hoy, ...","[59, 2046, 175, 13, 571, 0, 186, 15, 252, 164,..."
2,12553,"@victorita_fest para nada, ya arranque el segu...",0.0,"[arranqu, segund, term, veng, find, larg, da, ...","[11747, 237, 5749, 656, 1392, 487, 134, 3025, ..."
4,15619,@marianorajoy Va a explicar el programa o no ?...,1.0,"[va, explic, program, voy, vot, nunc, prestig]","[33, 294, 159, 50, 9, 119, 4693, 0, 0, 0, 0, 0..."
5,1851,RT @marcvidal: en 6 meses los españoles sacaro...,1.0,"[6, mes, español, sac, 54.000, millon, eur, es...","[436, 121, 92, 289, 0, 190, 240, 44, 0, 0, 0, ..."
...,...,...,...,...,...
3408,326,"Hoy la Ertzaintza ha descubierto un""ZULO""con a...",0.0,"[hoy, ertzaintz, descubiert, zul, armament, et...","[5, 4301, 0, 12918, 4120, 375, 909, 922, 0, 33..."
3409,167,"Sobre ""accidentes""... RT @santigonzalez10 Él h...",1.0,"[accident, ..., rt, lleg]","[1425, 0, 19, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."
3410,6931,"¿Cómo iba vestida? ""¿Cómo explicarlo? -me dice...",1.0,"[¿, com, iba, vest, ¿, com, explic, dic, luis,...","[6, 65, 666, 1648, 6, 65, 294, 26, 577, 822, 1..."
3412,9828,Me duele saber que en mis relaciones soy super...,1.0,"[duel, sab, relacion, sup, fiel, pendej, final...","[754, 30, 1072, 339, 1789, 1777, 167, 390, 18,..."


In [292]:
model.compile(
optimizer = "adam",
loss = "binary_crossentropy",
metrics = ["accuracy"]
)

In [301]:
results = model.fit(
x_train_tensor, train_binary.sentimiento,
epochs= 2,
batch_size = 32,
validation_data = (x_valid_tensor, valid_binary.sentimiento)
)

Epoch 1/2


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_19" is incompatible with the layer: expected axis -1 of input shape to have value 10000, but received input with shape (None, 27)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 27), dtype=int64)
  • training=True
  • mask=None

## RNN

In [300]:
# fixing every word's embedding size to be 32
embd_len = 32

# Creating a RNN model
RNN_model = Sequential(name="Simple_RNN")
RNN_model.add(Embedding(len(freq_dict),
						embd_len))

# In case of a stacked(more than one layer of RNN)
# use return_sequences=True
RNN_model.add(SimpleRNN(128,
						activation='tanh',
						return_sequences=False))
RNN_model.add(Dense(1, activation='sigmoid'))

# printing model summary
print(RNN_model.summary())

# Compiling model
RNN_model.compile(
	loss="binary_crossentropy",
	optimizer='adam',
	metrics=['accuracy']
)

# Training the model
history = RNN_model.fit(x_train_tensor, train_binary.sentimiento,
						batch_size=64,
						epochs=5,
						verbose=1,
						validation_data=(x_valid_tensor, valid_binary.sentimiento))

# Printing model score on test data
print()
print("Simple_RNN Score---> ", RNN_model.evaluate(x_test_tensor, test_binary.sentimiento, verbose=0))


Model: "Simple_RNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                         │          0 │
│                                 │                           │  (unbuilt) │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ simple_rnn_3 (SimpleRNN)        │ ?                         │          0 │
│                                 │                           │  (unbuilt) │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_23 (Dense)                │ ?                         │          0 │
│                                 │                           │  (unbuilt) │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.5283 - loss: 0.6911 - val_accuracy: 0.5257 - val_loss: 0.7046
Epoch 2/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7099 - loss: 0.5572 - val_accuracy: 0.5859 - val_loss: 0.7568
Epoch 3/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9099 - loss: 0.2265 - val_accuracy: 0.5688 - val_loss: 0.9882
Epoch 4/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9794 - loss: 0.0695 - val_accuracy: 0.5846 - val_loss: 1.2897
Epoch 5/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.9926 - loss: 0.0302 - val_accuracy: 0.5758 - val_loss: 1.4428

Simple_RNN Score--->  [1.409131646156311, 0.5799648761749268]


In [304]:
# Defining GRU model
gru_model = Sequential(name="GRU_Model")
gru_model.add(Embedding(len(freq_dict),
						embd_len))
gru_model.add(GRU(128,
				activation='tanh',
				return_sequences=False))
gru_model.add(Dense(1, activation='sigmoid'))

# Printing the Summary
print(gru_model.summary())

# Compiling the model
gru_model.compile(
	loss="binary_crossentropy",
	optimizer='adam',
	metrics=['accuracy']
)

# Training the GRU model
history2 = gru_model.fit(x_train_tensor, train_binary.sentimiento,
						batch_size=64,
						epochs=5,
						verbose=1,
						validation_data=(x_valid_tensor, valid_binary.sentimiento,))

# Printing model score on test data
print()
print("GRU model Score---> ", gru_model.evaluate(x_test_tensor, test_binary.sentimiento, verbose=0))


Model: "GRU_Model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ embedding_5 (Embedding)         │ ?                         │          0 │
│                                 │                           │  (unbuilt) │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ gru (GRU)                       │ ?                         │          0 │
│                                 │                           │  (unbuilt) │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_25 (Dense)                │ ?                         │          0 │
│                                 │                           │  (unbuilt) │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.5103 - loss: 0.6934 - val_accuracy: 0.5196 - val_loss: 0.6926
Epoch 2/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.5041 - loss: 0.6938 - val_accuracy: 0.5196 - val_loss: 0.6926
Epoch 3/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.5061 - loss: 0.6933 - val_accuracy: 0.5196 - val_loss: 0.6927
Epoch 4/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.5205 - loss: 0.6927 - val_accuracy: 0.5196 - val_loss: 0.6929
Epoch 5/5
107/107 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.5078 - loss: 0.6931 - val_accuracy: 0.5196 - val_loss: 0.6927

GRU model Score--->  [0.6928058862686157, 0.5079085826873779]


In [311]:
# Defining Bidirectional LSTM model
bi_lstm_model = Sequential(name="Bidirectional_LSTM")
bi_lstm_model.add(Embedding(len(freq_dict),
							embd_len))
bi_lstm_model.add(Bidirectional(LSTM(128,
									activation='tanh',
									return_sequences=False)))
bi_lstm_model.add(BatchNormalization())
bi_lstm_model.add(Dropout(0.3))
bi_lstm_model.add(Dense(1, activation='sigmoid'))

# Printing model summary
print(bi_lstm_model.summary())

# Compiling model summary
bi_lstm_model.compile(
loss="binary_crossentropy",
optimizer='adam',
metrics=['accuracy']
)

# Training the model
history4 = bi_lstm_model.fit(x_train_tensor, train_binary.sentimiento,
							batch_size=64,
							epochs=5,
							verbose=2,
							validation_data=(x_valid_tensor, valid_binary.sentimiento))

# Printing model score on test data
print()
print("Bidirectional LSTM model Score---> ",
	bi_lstm_model.evaluate(x_test_tensor, test_binary.sentimiento, verbose=0))

Model: "Bidirectional_LSTM"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ embedding_10 (Embedding)        │ ?                         │          0 │
│                                 │                           │  (unbuilt) │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ bidirectional_4 (Bidirectional) │ ?                         │          0 │
│                                 │                           │  (unbuilt) │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ batch_normalization_3           │ ?                         │          0 │
│ (BatchNormalization)            │                           │  (unbuilt) │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_15 (Dropout)            │ ?                         │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_30 (Dense)                │ ?                         │          0 │
│                                 │                           │  (unbuilt) │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/5
107/107 - 15s - 141ms/step - accuracy: 0.5913 - loss: 0.6650 - val_accuracy: 0.6167 - val_loss: 0.6876
Epoch 2/5
107/107 - 7s - 67ms/step - accuracy: 0.7958 - loss: 0.4609 - val_accuracy: 0.6088 - val_loss: 0.6742
Epoch 3/5
107/107 - 7s - 65ms/step - accuracy: 0.9000 - loss: 0.2592 - val_accuracy: 0.5455 - val_loss: 0.7121
Epoch 4/5
107/107 - 7s - 64ms/step - accuracy: 0.9490 - loss: 0.1546 - val_accuracy: 0.5820 - val_loss: 0.7840
Epoch 5/5
107/107 - 7s - 65ms/step - accuracy: 0.9667 - loss: 0.1068 - val_accuracy: 0.5776 - val_loss: 1.1549

Bidirectional LSTM model Score--->  [1.1895959377288818, 0.5702987909317017]
